In [44]:
import sys
import quandl
import pandas
import numpy as np
import tensorflow as tf

quandl.ApiConfig.api_key = 'RUQDgag1PxTWxCVRgiBB'

# stock ndexes
item_list = ["YAHOO/INDEX_GSPC","YAHOO/INDEX_DJI","YAHOO/INDEX_IXIC","YAHOO/INDEX_NYA","YAHOO/INDEX_XAX","YAHOO/INDEX_RUT","YAHOO/INDEX_VIX","YAHOO/INDEX_GDAXI","YAHOO/INDEX_FCHI","YAHOO/INDEX_STOXX50E","YAHOO/INDEX_BFX","YAHOO/INDEX_N225","YAHOO/INDEX_HSI","YAHOO/INDEX_STI","YAHOO/INDEX_AXJO","YAHOO/INDEX_AORD","YAHOO/INDEX_BSESN","YAHOO/INDEX_JKSE","YAHOO/INDEX_NZ50","YAHOO/INDEX_KS11","YAHOO/INDEX_TWII","YAHOO/INDEX_GSPTSE","YAHOO/INDEX_BVSP","YAHOO/INDEX_MXX","YAHOO/INDEX_MERV","YAHOO/INDEX_TA100"]
   
# function
def get_formatted_data(start_date, end_date, estimate_day):
    # get stock data
    for i in range(len(item_list)):
        data = quandl.get(item_list[i], start_date=start_date, end_date=end_date)
        data = data.loc[:,"Adjusted Close":"Adjusted Close"]
        data.columns = [item_list[i]]
        if i == 0:
            stock_df = data
        else:
            stock_df = pandas.concat([stock_df, data], axis=1)

    # get gold data and shift
    gold_df = quandl.get("WGC/GOLD_DAILY_KRW", start_date = start_date, end_date = end_date)
    shifted_gold_df = gold_df.shift(periods = -1 * estimate_day)

    # merge
    merged_df =  pandas.concat([stock_df, shifted_gold_df], axis=1)
    merged_df = merged_df.dropna(axis=1, how = "all")
    merged_df = merged_df.dropna(axis=0, how = "any")
    
    # change data into tensorflow sytle
    num_of_sample = len(merged_df)
    num_of_stocks = len(merged_df.columns) - 1
    merged_np = merged_df.as_matrix()
    stock_np = merged_np[:,:num_of_stocks]
    gold_np = merged_np[:,num_of_stocks:]
    new_gold_np = []
    for i in range(len(gold_np)):
        if gold_np[i][0] >= 0:
            new_gold_np.append([1, 0])
        else:
            new_gold_np.append([0, 1])

    return stock_np, new_gold_np

# parameter
estimate_day = 1

# get train data
train_start_date = "2014-01-01"
train_end_date = "2016-12-31"
stock_train, gold_train = get_formatted_data(train_start_date, train_end_date, estimate_day)
if len(stock_train) == 0:
    print("No Validation Data")
    exit()
else:
    print("Trainning Data: ", len(stock_train))
    
# get valdaition data
valid_start_date = "2017-01-01"
valid_end_date = "2017-02-20"
stock_valid, gold_valid = get_formatted_data(valid_start_date, valid_end_date, estimate_day)
if len(stock_valid) == 0:
    print("No Validation Data")
else:
    print("Validation Data: ", len(stock_valid))

('Trainning Data: ', 509)
('Validation Data: ', 20)


In [45]:
#Plot settings
training_epochs = 25
learning_rate = 0.1
display_step = 1

# train and validation
x = tf.placeholder("float", [None, len(stock_train[0])])
y = tf.placeholder("float", [None, 2]) # increase or decrease

# Set model weights
W = tf.Variable(tf.zeros([len(stock_train[0]), 2]))
b = tf.Variable(tf.zeros([2]))

# Construct model
activation = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cross_entropy = y * tf.log(activation)
cost = tf.reduce_mean(-tf.reduce_sum(cross_entropy, reduction_indices = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [46]:
batch_size = 25

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training cycle
    avg_cost = 0.
    total_batch = int(len(gold_train) / batch_size)
    for i in range(total_batch):
        stock_batch = stock_train[i * batch_size : (i + 1) * batch_size]
        gold_batch = gold_train[i * batch_size : (i + 1) * batch_size]
        # Compute average loss
        avg_cost += sess.run(cost, { x: stock_train[i * batch_size : (i + 1) * batch_size], y: gold_train[i * batch_size : (i + 1) * batch_size] }) / total_batch
        # Display logs per epoch step
        print("batch: %04d" % (i + 1), "cost= {:.9f}".format(avg_cost))
    print("Training phase finished")

    # Test model
    correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Model accuracy:", accuracy.eval({ x: stock_valid, y: gold_valid }))

('batch: 0001', 'cost= 0.034657359')
('batch: 0002', 'cost= 0.069314718')
('batch: 0003', 'cost= 0.103972077')
('batch: 0004', 'cost= 0.138629436')
('batch: 0005', 'cost= 0.173286796')
('batch: 0006', 'cost= 0.207944155')
('batch: 0007', 'cost= 0.242601514')
('batch: 0008', 'cost= 0.277258873')
('batch: 0009', 'cost= 0.311916232')
('batch: 0010', 'cost= 0.346573591')
('batch: 0011', 'cost= 0.381230950')
('batch: 0012', 'cost= 0.415888309')
('batch: 0013', 'cost= 0.450545669')
('batch: 0014', 'cost= 0.485203028')
('batch: 0015', 'cost= 0.519860387')
('batch: 0016', 'cost= 0.554517746')
('batch: 0017', 'cost= 0.589175105')
('batch: 0018', 'cost= 0.623832464')
('batch: 0019', 'cost= 0.658489823')
('batch: 0020', 'cost= 0.693147182')
Training phase finished
('Model accuracy:', 1.0)
